In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd gdrive/MyDrive/DISSERTATION/SFC-CAE-AdaptiveFinal/

[Errno 2] No such file or directory: 'gdrive/MyDrive/DISSERTATION/SFC-CAE-AdaptiveFinal/'
/content/gdrive/MyDrive/DISSERTATION/SFC-CAE-AdaptiveFinal


In [ ]:
# !git pull

### Install dependencies

In [ ]:
modulelist = !pip list                         #This is just to check whether the requirements are already installed
if str(modulelist).find("vtk") == -1:          #If vtk is already installed don't start the installation process because it will take a couple minutes to verify that everything is there.
  !pip install -e .

### Import useful functions inside this repo, very important, don't forget!!!

In [ ]:
from sfc_cae import *

# Specify the folder of the data and load the full_tensors from `vtu` files, or `txt` files, like following, check the folder you specified only contains `vtu` and `txt` files!!!

### specify your own {DATA_DIR} of vtu files here.

In [ ]:
# data_path = 'FPC_Re3900_CG_new/'
# vtu_fields = ['Velocity']

### Specify the `vtu_fields` (a list) of an unadapted mesh to read in, returns:
**full_tensor**: The tensor of all `vtu_fields` you specified for all snapshots in the {data_path}, 3-dimension, the first dimension is number of snapshots in that {data_path}, the second is the number of Nodes in each snapshots, the last is number of components in the `vtu_fields`, for example, in this FPC test case,
two components for `Velocity`, one for `Pressure`. <br> <br>
**coords**: The coordinates of the unadapted mesh Nodes from the `vtu` file, 3-dimension.

In [ ]:
# full_tensor, coords, cells = read_in_files(data_path, vtu_fields = vtu_fields)

For reading tensor from `txt` files, check the `read_in_files` function from `util.py` at https://github.com/acse-jy220/SFC-CAE-Ready-to-use

### Save the full tensor in your `google_drive` for further use.

In [ ]:
# torch.save(full_tensor, '/content/gdrive/MyDrive/FPC_CG_new_velocity.pt')
# torch.save(coords, '/content/gdrive/MyDrive/FPC_CG_new_mesh.pt')

### Load tensor from google drive

In [ ]:
# full_tensor = torch.load('./savedtensors/FPC_new_velocity.pt')[1:]
# coords = torch.load('./savedtensors/FPC_new_mesh.pt')
# cells = torch.load('./savedtensors/FPC_new_mesh_cells.pt')

In [ ]:
# import pickle
# pickle_in = open("adaptivereadytotrain.pickle","rb")
# full_tensor = pickle.load(pickle_in)

In [1]:
# #We create a list of ordered and then interpolated tensors
# interpolated = []
# for i in range(len(full_tensor)):
#   for j in range(10):
#     concl = full_tensor[i][:2,full_tensor[i][4+j].long()]
#     xl = full_tensor[i][2:4,full_tensor[i][4+j].long()]
#     a, b = interpolate(xl,concl,6000)
#     x_l = a
#     conc_l = b
#     # print(x_l, conc_l, full_tensor[i][j].long())
#     interpolated.append(np.concatenate((conc_l,x_l),0))
#   del x_l
#   del conc_l

In [ ]:
# full_tensor = torch.Tensor(interpolated)

In [ ]:
# torch.save(full_tensor, "Interpoltensor_backup.pth")

In [ ]:
full_tensor = torch.load("Interpoltensor_backup.pth")

FileNotFoundError: ignored

### Random split the tensor into training_set: valid_set: test_set = 0.8:0.1:0.1 = 8:1:1

In [ ]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1
train_index, valid_index, test_index = index_split(train_ratio, valid_ratio, test_ratio, total_num = 18000)

In [ ]:
train_set = full_tensor[train_index-1]
valid_set = full_tensor[valid_index-1]
test_set = full_tensor[test_index-1]

In [ ]:
# torch.save(train_set,"./savedtensors/FPC_interpol_adaptive_train.pt")
# torch.save(valid_set,"./savedtensors/FPC_interpol_adaptive_valid.pt")
# torch.save(test_set,"./savedtensors/FPC_interpol_adaptive_test.pt")

In [ ]:
# train_set = torch.load("./savedtensors/FPC_interpol_adaptive_train.pt")
# valid_set = torch.load("./savedtensors/FPC_interpol_adaptive_valid.pt")
# test_set = torch.load("./savedtensors/FPC_interpol_adaptive_test.pt")

### The following cell is quite vital, as it indicates the parameters for the **autoencoder**

`input_size` is the number of Node, for example 20550 <br> `dimension` is the dimension of the data in Vtu file, 2 or 3.<br> `components` is the number of components you are compressing, if the tensor is 2 dimension, then it's default of 1. `self_concat` is chennel copying operation, you could keep it default as 1, or 2 when you have multiple SFCs. <br> `structured`: bool, whether you are training SFC-CAE on a structured mesh or not.<br>`nearest_neighboring`: bool, adding sparse layers or not, suggest to turn on.<br>
`dims_letent`: The dimension of the compressed data after applying decoder. <br>
`space_filling_orderings`, `invert_space_filling_orderings` : lists of sfc\inverse_sfc nums, generated by `get_sfc_curves_from_coords` function, or read the orderings from a `csv` file on your own. <br>
`activation`: Can be customly defined, but if you not spcify, the activation would be `ReLU()` for structured mesh, and `Tanh()` for unstructured mesh.

### The following are just normal Hyperparameters for training, free to tune it yourself.

In [ ]:
batch_size = 64 #Unfortunately the batch size is going to have to be 1 yet again
seed = 27
lr = 1e-4
n_epochs = 2000
weight_decay = 0
criterion_type = 'MSE'
valid_batch_size = 200

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=valid_batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=valid_batch_size, shuffle=True)

In [ ]:
fakesfc = [np.array(range(6000))]  *2 #Creating a "fake" space filling curve to use the normal structure of the autoencoder

In [ ]:
fakeisfc = [np.array(range(6000))] *2

In [ ]:
input_size = 6000
components = 2
self_concat = 1
structured = False
nearest_neighbouring = True #True # shut down this may lead to a slightly poorer performance, but a quicker training spped
dims_latent = 64
activation = nn.Tanh()
variational = False
dimension = 2

In [ ]:
autoencoder = SFC_CAE_interpol(input_size,
                      dimension,
                      components,
                      structured,
                      self_concat,
                      nearest_neighbouring,
                      dims_latent,
                      fakesfc, 
                      fakeisfc,
                      activation,
                      variational = variational,
                      force_initialising_param=None,
                      coption = 1,
                      coordslayers = [1,0],
                      verbose = False,
                      nfclayers = 2,
                      smoothinglayers = [[(8,33)],[]])

In [ ]:
autoencoder.encoder.orderings[0][autoencoder.decoder.orderings[0]]

In [ ]:
autoencoder

In [ ]:
# autoencoder.load_state_dict(torch.load("toloadinterpolation.pth"))

In [ ]:
autoencoder = train_model(autoencoder, 
                          train_loader = train_loader,
                          valid_loader = valid_loader,
                          test_loader = test_loader,
                          state_load = None,
                          n_epochs = 4000, 
                          lr = 1e-4,
                          shuffle = False,
                          weight_decay = weight_decay,
                          criterion_type = criterion_type,
                          seed = seed,
                          optimizer = 'Adam',
                          sfcstoshuffle = [np.array(range(6000))],
                          isfcstoshuffle = [np.array(range(6000))],
                          interpol = True,
                          save_path = "./")

### Other functions need to be add, like convert the result back to `csv`/`vtu` files.....

In [ ]:
# torch.save(autoencoder.state_dict(), "0.007adaptive2.pth")

In [ ]:
autoencoder.load_state_dict(torch.load("0.007adaptive2.pth"))

In [ ]:
full_tensor[19000]

In [ ]:
batch = full_tensor[5].unsqueeze(0)
batch = batch.permute(0,2,1).to("cuda")
result = autoencoder(batch)
point_data = {}
point_data.update({"Velocity":result[0].permute(1,0).cpu().detach().numpy()})
mesh = meshio.Mesh(batch[:,:,2:4].cpu().detach().numpy(),point_data)
mesh.write("./UnseenInterpol.vtu")

In [ ]:
result_vtu_to_vtu(data_path, save_path, vtu_fields, autoencoder, k.detach().numpy(), b.detach().numpy(), model_device = model_device)

Please find the reconstructed files in `reconstructed` folder